Read in

In [300]:
# To generate and store data.
import numpy as np
import pandas as pd

# To visualize data.
import matplotlib.pyplot as plt

# To fit linear regression model.
from sklearn.linear_model import LinearRegression

In [301]:
assistance_df = pd.read_csv('HazardMitigationAssistanceMitigatedProperties.csv')

Change Cols

In [302]:
assistance_df.columns = assistance_df.columns.str.lower()
assistance_df.columns = assistance_df.columns.str.replace(' ', '_')
assistance_df.columns = assistance_df.columns.str.strip()

Drop Cols

In [303]:
assistance_df = assistance_df.drop(columns=['tribalindicator', 'projectidentifier','status','programarea', 'type', 'region', 'statenumbercode', 'countycode'])

In [304]:
assistance_df = assistance_df.drop(columns=['dateinitiallyapproved','dateclosed'])

Convert Dates

In [306]:
# Remove all characters after the date

assistance_df['dateapproved'] = assistance_df['dateapproved'].str[:10]



Fill Easy NaNs

In [307]:
assistance_df['disasternumber'] = assistance_df['disasternumber'].fillna(0)
assistance_df['actualamountpaid'] = assistance_df['actualamountpaid'].fillna(0)
assistance_df['damagecategory'] = assistance_df['damagecategory'].fillna('')
assistance_df['title'] = assistance_df['title'].fillna('')

Fill Dates with Date formatted Year (assume YYYY-01-01)

In [308]:
assistance_df.loc[assistance_df['dateapproved'].isnull()] = str(assistance_df['programfy']) + '-01-01'

Check State

In [309]:
state_nan_indices = assistance_df.loc[assistance_df['state'].isnull()].index

In [310]:
zips_for_state_nans = []

for nan_index in state_nan_indices:
    zips_for_state_nans.append(assistance_df.iloc[nan_index].zipcode)
    
    

In [311]:
states_for_zips = []

for zip in zips_for_state_nans:
    
    try:
        state_fill_in = assistance_df.loc[assistance_df['zipcode'] == zip, 'state'].value_counts().index[0]
        states_for_zips.append(state_fill_in)
    except:
        states_for_zips.append('')
    

In [312]:
zips_for_state_nans[10] = ''

In [313]:
state_fill_df =pd.DataFrame({ 'indices': list(state_nan_indices), 'state': states_for_zips, 'zip' : zips_for_state_nans })

In [314]:
state_fill_df.loc[state_fill_df['state'] == '', 'state'] = 'Virginia'

In [315]:
state_fill_df

,indices,state,zip
0,2181,West Virginia,25984
1,3385,Virginia,24279
2,5655,Mississippi,38829
3,14251,California,95111
4,18865,North Dakota,58237
5,21392,Kentucky,40351
6,21681,Virginia,02470
7,21741,Oklahoma,74546
8,22143,Oregon,97420
9,23355,Louisiana,70087


In [316]:
for i in range(state_fill_df.shape[0]):
    assistance_df.iloc[state_fill_df.iloc[i,0],1] = state_fill_df.iloc[i,1]

Cities

In [317]:
assistance_df.loc[assistance_df['city'].isnull()]

,disasternumber,state,county,city,zipcode,propertyaction,structuretype,typeofresidency,actualamountpaid,programfy,dateapproved,title,numberofproperties,damagecategory
28238,1347,Virginia,NaN,NaN,NaN,Acquisition,Single Family,Owner Occupied - Principal Residence,90000,2001,2007-05-23,Aguila Mitigation Project - Acquisition,1,
55822,0,Washington,Thurston,NaN,98502,Other,Other (Specify in Comments),NaN,0,2005,2006-07-22,Independence Road/Chehalis River Flood Mitigat...,1,


In [319]:
assistance_df.iloc[55822,3] = 'Olympia'
assistance_df.iloc[28238,3] = ''

Zipcode

In [320]:
list_of_cities_for_zip = list(assistance_df.loc[assistance_df['zipcode'].isnull(),'city'])

In [321]:
assistance_df.loc[assistance_df['zipcode'].isnull()]

for city in list_of_cities_for_zip:
    try:
        zipcode_filler = assistance_df.loc[assistance_df['city'] == city,'zipcode'].mode()[0]
        print(city, zipcode_filler)
    except:
        print(city, '')

CAPE GIRARDEAU 63701
LINCOLN COUNTY 6....
CARROLLTON 35447
 
FRANKLIN CO 
FRANKLIN 46131
BLUE SPRINGS 38828
APPLE VALLEY 92392


In [322]:
assistance_df.iloc[5917,4] = 63701
assistance_df.iloc[6900,4] = 63343
assistance_df.iloc[32389,4] = 46131
assistance_df.iloc[33388,4] = 46131
assistance_df.iloc[34041,4] = 38828
assistance_df.iloc[56122,4] = 92392
assistance_df.iloc[15696, 4] = 64622
assistance_df = assistance_df.drop(28238)

Nominal Data

In [328]:
assistance_df.isnull().sum()

disasternumber            0
state                     0
county                 1046
city                      0
zipcode                   0
propertyaction         8543
structuretype             0
typeofresidency       11988
actualamountpaid          0
programfy                 0
dateapproved              0
title                     0
numberofproperties        0
damagecategory            0
dtype: int64

In [329]:
assistance_df.to_csv('disaster_support.csv', index=False)